In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import h5py
import glob
import tqdm
np.set_printoptions(suppress=True)

In [ ]:
total_path = r'Z:\zym\IS\TH_chr2_5'
PPR_path = total_path + '\\'+'PPR'+'\\'+'plx'
filelist_p = os.listdir(PPR_path)
plx_p=[]
for f in filelist_p:
    if f.endswith('.plx'):
        plx_p.append(f[:-4])

In [ ]:
ptd_all = []
wave_data = []
unit_name = []
for idx,file_name in enumerate(plx_p):
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    wave = wave[wave.peak_trough_distance != 0]
    wave_data.extend(wave.values[:,0:3].tolist())
    unit_name.extend((wave.values[:,0]).tolist())
unit_name = list(dict.fromkeys(unit_name))
unit_name.sort()
for u in unit_name:
    ptd = []
    for w in wave_data:
        if w[0] == u:
            ptd.append(w[2])
    ptd_all.append(np.mean(ptd))
ptd_avg = np.array(ptd_all)
narrow_idx = np.argwhere((ptd_avg<0.4)|(ptd_avg==0.4)).flatten()
narrow_idx = narrow_idx.tolist()
broad_idx = np.argwhere(ptd_avg>0.4).flatten()
broad_idx = broad_idx.tolist()

In [ ]:
unit_narrow = []
for i in narrow_idx:
    unit_narrow.append(unit_name[i])
unit_broad = []
for i in broad_idx:
    unit_broad.append(unit_name[i])

In [ ]:
wave_p = pd.read_excel(os.path.join(PPR_path, plx_p[0]+'waveform.xlsx'),engine='openpyxl')

In [ ]:
unit_ppr = []
ppr_df = pd.DataFrame(columns=['day1','day2','day3'])
for file_name in plx_p:
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    wave = wave[wave.peak_trough_distance != 0]
    wave_data.extend(wave.values[:,0:3].tolist())
    unit_ppr.extend((wave.values[:,0]).tolist())
unit_ppr = list(dict.fromkeys(unit_ppr))
unit_ppr.sort()

In [ ]:
ppr_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
ppr_pp_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
#ppr_idv_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
kbd = np.arange(0,30,3.31)
gap = 1155/1000
cal_gap = 500/1000
for idx,file_name in enumerate(plx_p):
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    timestamp = h5py.File(os.path.join(PPR_path, file_name+'timestamp.mat'),'r')
    kbd_p = timestamp['timestamp/kbd1'][:].flatten()
    kbd_p_list = kbd_p + kbd
    unit_list = list(wave.unit)
    unit_name = []
    for i in unit_list:
        name = 'timestamp/'+i
        unit_name.append(name)
        spike_p = timestamp[name][:].flatten()
        ppr_10trial = []
        ppr_p1_p2 = []
        for k_p in kbd_p_list:
            start1 = k_p + 5/1000
            stop1 = start1 + cal_gap
            start2 = start1 + gap
            stop2 = start2 + cal_gap
            #print([start1,stop1,start2,stop2])
            spk_array1 = spike_p[(spike_p>start1) & (spike_p<stop1)]
            spk_array2 = spike_p[(spike_p>start2) & (spike_p<stop2)]
            if len(spk_array1)/cal_gap != 0:
                ppr = (len(spk_array2)/cal_gap)/(len(spk_array1)/cal_gap)
            elif (len(spk_array1)==0) & (len(spk_array2)==0):
                ppr = 1
            else:
                ppr = 0
            ppr_10trial.append(ppr)
            ppr_p1_p2.append([len(spk_array1)/cal_gap,len(spk_array2)/cal_gap])
        ppr = np.mean(ppr_10trial)
        ppr_pp = np.mean(np.array(ppr_p1_p2),0)
        #ppr_10trial = [float('{:.3f}'.format(i)) for i in ppr_10trial]
        ind = 'day%s'% (idx+1)
        ppr_pp_df.loc[ind, i] = ppr_pp
        ppr_df.loc[ind, i] = ppr_pp[1]/ppr_pp[0]
        #ppr_idv_df.loc[ind, i] = ppr_10trial
        if i in unit_broad:
            ppr_df.loc['cell_type',i] = 'broad'
            ppr_pp_df.loc['cell_type',i] = 'broad'
            #ppr_idv_df.loc['cell_type',i] = 'broad'
        elif i in unit_narrow:
            ppr_df.loc['cell_type',i] = 'narrow'
            ppr_pp_df.loc['cell_type',i] = 'narrow'
            #ppr_idv_df.loc['cell_type',i] = 'narrow'
        else:
            ppr_df.loc['cell_type',i] = 'none'
            ppr_pp_df.loc['cell_type',i] = 'none'
            #ppr_idv_df.loc['cell_type',i] = 'none'
ppr_df.fillna(0,inplace=True)
ppr_df.to_excel(os.path.join(PPR_path,'ppr.xlsx'))
ppr_pp_df.to_excel(os.path.join(PPR_path,'ppr_p1_p2.xlsx'))
#ppr_idv_df.to_excel(os.path.join(PPR_path,'ppr_10trial.xlsx'))

In [ ]:
ppr_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
gap = 1155/1000 
cal_gap = 500/1000
ppr_3day = []
pre_3day = []
post_3day = []
for idx,file_name in enumerate(plx_p):
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    timestamp = h5py.File(os.path.join(PPR_path, file_name+'timestamp.mat'),'r')
    kbd_p = timestamp['timestamp/kbd1'][:].flatten()
    start1 = kbd_p + 5/1000
    stop1 = start1 + cal_gap
    start2 = start1 + gap
    stop2 = start2 + cal_gap
    print([start1,stop1,start2,stop2])
    ppr_all = []
    unit_list = list(wave.unit)
    unit_name = []
    pre_all = []
    post_all = []
    
    for i in unit_list:
        name = 'timestamp/'+i
        unit_name.append(name)
        spike_p = timestamp[name][:].flatten()
        spk_array1 = spike_p[(spike_p>start1) & (spike_p<stop1)]
        spk_array2 = spike_p[(spike_p>start2) & (spike_p<stop2)]
        if len(spk_array1)/cal_gap != 0:
            ppr = (len(spk_array2)/cal_gap)/(len(spk_array1)/cal_gap)
        elif (len(spk_array1)==0) & (len(spk_array2)==0):
            ppr = 1
        else:
            ppr = 0
        ind = 'day%s'% (idx+1)
        ppr_df.loc[ind, i] = ppr
        if i in unit_broad:
            ppr_df.loc['cell_type',i] = 'broad'
        else:
            ppr_df.loc['cell_type',i] = 'narrow'
        ppr_all.append(ppr)
        pre_all.append(len(spk_array1)/cal_gap)
        post_all.append(len(spk_array2)/cal_gap)
    ppr_3day.append(ppr_all)
    post_3day.append(post_all)
    pre_3day.append(pre_all)
ppr_3day = np.array(ppr_3day)
post_3day = np.array(post_3day)
pre_3day = np.array(pre_3day)
pre_n_post = np.hstack((pre_3day,post_3day))

In [ ]:
ppr_df.fillna(0,inplace=True)
ppr_df.to_excel(os.path.join(PPR_path,'ppr_1trial.xlsx'))